# Image Registration and Combination using the JWST Level 3 Pipeline - MIRI example

Stage 3 imaging (Image3, calwebb_image3) processing is intended for combining the calibrated data from multiple exposures (e.g., a dither or mosaic pattern) into a single distortion corrected product. Before being combined, the exposures receive additional corrections for the purpose of astrometric alignment, background matching, and outlier rejection. 

> **Inputs**: The inputs to calwebb_image3 will usually be in the form of an association (ASN) file that lists multiple associated 2D calibrated exposures to be processed and combined into a single product. The individual exposures should be calibrated ("cal") from calwebb_image2 processing. It is also possible use a single "cal" file as input, in which case only the resample and source_catalog steps will be applied.

> **Outputs**: A resampled/rectified 2D image product with suffix "i2d" is created, containing the rectified single exposure or the rectified and combined association of exposures (the direct output of the resample step). A source catalog produced from the "i2d" product is saved as an ASCII file in "ecsv" format, with a suffix of "cat". If the outlier_detection step is applied, a new version of each input calibrated exposure product is created, which contains a DQ array that has been updated to flag pixels detected as outliers. This updated product is known as a CR-flagged product and the file is identified by including the association candidate ID in the original input "cal" file name and changing the suffix to "crf".
    

Level 3 pipeline steps:

**Tweakreg** (jwst.tweakreg, tweakreg_step, TweakRegStep)

**Sky Match** (jwst.skymatch, skymatch_step, SkyMatchStep)

**Outlier Detection** (jwst.outlier_detection, outlier_detection_step, OutlierDetectionStep)

**Resample** (jwst.resample, resample_step, ResampleStep)

**Source Catalog** (jwst.source_catalog, source_catalog_step, SourceCatalogStep)

(for more information on individual steps see: https://jwst-pipeline.readthedocs.io/en/latest/jwst/package_index.html)

### Table of Contents:
> * [Resources and Documentation](#resources)
> * [Create Association table](#association)
> * [Using Configuration Files](#pipeline_configs)
> * [Run Pipeline with Configuration Files](#pipeline_with_cfgs)
> * [Run Pipeline with Paramters Set Programmatically](#pipeline_no_configs)
> * [Run Individual Steps with Configuration Files](#steps_with_config_files)
> * [Run Individual Steps with Parameters Set Programmatically](#steps_no_configs)

***
<a id='resources'></a>
## 1. Resources and Documentation

There are several different places to find information on installing and running the pipeline. This notebook will give a shortened description of the steps pulled from the detailed pipeline information pages, but to find more in-depth instructions use the links below. 

>1. JDox: https://jwst-docs.stsci.edu/display/JDAT/JWST+Data+Reduction+Pipeline
>2. Installation page: http://astroconda.readthedocs.io/en/latest/releases.html#pipeline-install
>3. Detailed pipeline information: https://jwst-pipeline.readthedocs.io/en/latest/jwst/introduction.html
>4. Help Desk (click on Pipeline Support): https://stsci.service-now.com/jwst?id=sc_category
>5. GitHub README installation instructions: https://github.com/spacetelescope/jwst/blob/master/README.md


If this is your first time trying to run the pipeline from a jupyter notebook, you need to install the jupyer notebook in your pipeline environment:
>1. In a new terminal, change the directory to your working directory, terminal command: cd [your woring directory]
>2. Terminal command: source activate jwst_dev
(or whatever your environment name for the pipeline is)
>3. Terminal command: conda install jupyter
>4. Terminal command: jupyter notebook


In [ ]:
from glob import glob
from astropy.io import fits
from astropy.utils.data import download_file
import matplotlib.pyplot as plt
%matplotlib inline
from jwst import datamodels
# from jwst.associations.asn_from_list import asn_from_list  # perhaps can be done in the future

In [ ]:
from jwst.pipeline import Image3Pipeline

In [ ]:
url = 'https://stsci.box.com/shared/static/2vlo7yqk00wmpu8x32ipg127i8lynpr2.json'
import urllib.request
r = urllib.request.urlopen(url)
association_file = r.info().get_filename()

***
<a id='association'></a>
## 2. Create an Association Table

An association table is a **json** file that should contain all of the files to be combined in a single mosaic. Files that cannot be combined (e.g. NIRCam shortwave and longwave data) must be placed in separate association tables.

### An example association table

<a id='pipeline_configs'></a>
## 3. Using Configuration Files

Configuration files are optional inputs for each step of the pipeline, as well as for the pipeline itself. These files list step-specific parameters, and can also be used to control which steps are run as part of the pipeline.

You can get the full compliment of configuration files using the `collect_pipeline_cfgs` convenience function from the command line:

>`$ collect_pipeline_cfgs ./`

This creates a copy of all configuration files, for all steps and all JWST Instruments. Note that default parameters in the config files are not necessarily optimized for any particular instrument. 

Each of these configuration files can be customized to control pipeline behavior. For example, the configuration file for the Level 3 imaging pipeline is called **calwebb_image3.cfg** and contains a list (not necessarily in order) of the steps run as part of the Level 3 imaging pipeline.


    name = "Image3Pipeline"
    class = "jwst.pipeline.Image3Pipeline"

        [steps]
          [[tweakreg]]
            config_file = tweakreg.cfg
            skip = True
          [[skymatch]]
            config_file = skymatch.cfg
          [[outlier_detection]]
            config_file = outlier_detection.cfg
          [[resample]]
            config_file = resample.cfg
          [[source_catalog]]
            config_file = source_catalog.cfg
            save_results = true
        
In this example, the ***tweakreg*** step will be skipped (`skip = True`), and the output from the ***source_catalog*** step will be saved (`save_results = True`).

Note tht **calwebb_image3.cfg** lists a configuration file for each pipeline step. You can customize a particular pipeline step by editing the parameters in its configuration file. For example, the source catalog configuration file, shown below, contains details on the kernel size and FWHM, as well as the signal to noise threshold to use in the identification of sources in the final combined image. 


    name = "source_catalog"
    class = "jwst.source_catalog.SourceCatalogStep"

    kernel_fwhm = 3.
    kernel_xsize = 5.
    kernel_ysize = 5.
    snr_threshold = 3.
    npixels = 50
    deblend = False

## 3.5 Running the pipeline on MIRI data

The dataset being used in this notebook was provided by Misty Cracraft. From Misty:
"I have a set of four files I’ve worked on with 5 point sources, two files each at two different dither positions. The files do not have an APT file, but they can be combined by running them through the pipeline. The final output catalog has one extra position listed, if everything is run with defaults. The files can be found in /grp/jwst/ins/simdata/miri/5star_imager. I’ve gone ahead and added the rate files (output of calwebb_detector1) and the cal files (output of calwebb_image2) to the directory, as well as the files used to create the simulations in case those are helpful."

The files can also be found here: https://stsci.box.com/s/to6mcfmyap8kn7z9ordmcyb1dcbh1ps2. 

The association file is 'det_dithered_5stars.json' and has the following content:

The combined image is exported as: det_dithered_5stars_tweak_i2d.fits

***
<a id="pipeline_with_cfgs"></a>
## 4. Run Pipeline with Configuration Files

Once you have edited the configuration files to customize the Level 3 pipeline, the command below will run the pipeline.

This will generate a final source catalog ***cat.ecsv***, a final 2D image ***i2d.fits***, individual exposures with DQ array flagged for outliers ***crf.fits***, and blot images from the outlier detection step ***blot.fits***.

In [ ]:
m = Image3Pipeline.call(association_file, config_file='config/calwebb_image3.cfg')

### Examine Outputs

#### Combined Image

In [ ]:
# Output combined image
combined_image_file = 'det_dithered_5stars_tweak_i2d.fits'
combined_image = fits.getdata(combined_image_file)

In [ ]:
from astropy.visualization import LogStretch, ImageNormalize, ManualInterval
norm = ImageNormalize(combined_image, interval=ManualInterval(vmin=-25, vmax=25), stretch=LogStretch())
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1, 1, 1)
im = ax.imshow(combined_image, origin='lower', norm=norm)
fig.colorbar(im)
plt.show()

***
<a id="pipeline_no_configs"></a>
## 5. Run Pipeline with Parameters Set Programmatically

You can also run the pipeline without relying on configuration files by setting parameters programmatically, and relying on the defaults in the pipeline.

In [ ]:
m = Image3Pipeline()

# You can skip steps and change parameter values
m.tweakreg.skip = False
m.source_catalog.snr_threshold = 10
# run the pipeline with these paramters
m.run(association_file)

#### Combined Image

In [ ]:
from astropy.visualization import LogStretch, ImageNormalize, ManualInterval
combined_image_file = 'det_dithered_5stars_tweak_i2d.fits' ## need to load data gain
combined_image = fits.getdata(combined_image_file) ## need to load data again
norm = ImageNormalize(combined_image, interval=ManualInterval(vmin=-25, vmax=25), stretch=LogStretch())
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1, 1, 1)
im = ax.imshow(combined_image, origin='lower', norm=norm)
fig.colorbar(im)
plt.show()

***
<a id="steps_with_config_files"></a>
## 6. Run Individual Steps with Configuration Files

In [ ]:
from jwst.tweakreg import tweakreg_step
from jwst.skymatch import skymatch_step
from jwst.outlier_detection import outlier_detection_step
from jwst.resample import resample_step
from jwst.source_catalog import source_catalog_step

In [ ]:
m = tweakreg_step.TweakRegStep.call(association_file, config_file='config/tweakreg.cfg')
m = skymatch_step.SkyMatchStep.call(m, config_file='config/skymatch.cfg')
m = outlier_detection_step.OutlierDetectionStep.call(m, config_file='config/outlier_detection.cfg')
m = resample_step.ResampleStep.call(m, config_file='config/resample.cfg', output_file='jw10002_short_step_by_step_i2d.fits')
m = source_catalog_step.SourceCatalogStep.call(m, config_file='config/source_catalog.cfg', output_file='jw10002_short_step_by_step_cat.ecsv')

In [ ]:
from astropy.visualization import LogStretch, ImageNormalize, ManualInterval
combined_image_file = 'det_dithered_5stars_tweak_i2d.fits' ## need to load data gain
combined_image = fits.getdata(combined_image_file) ## need to load data again
norm = ImageNormalize(combined_image, interval=ManualInterval(vmin=-25, vmax=25), stretch=LogStretch())
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(1, 1, 1)
im = ax.imshow(combined_image, origin='lower', norm=norm)
fig.colorbar(im)
plt.show()